# Transformer

## Long-term dependency problem

- 기존에 연속된 데이터를 처리하기 위해서 사용하던 Model은 Recurrent Neural Network 였다.
  - Recurrent Neural Network는 이전 state의 정보를 이용해서 결과를 도출하기 때문에 현재 상태가 이전 정보의 영향을 받는 데이터라면 많이 사용하던 Model이다.
- 하지만, Recurrent Neural Network의 가장 큰 문제는 Long-term dependency 문제가 있다는 것이다.

- Long-term dependency란?
  - 어떤 정보와 다른 정보 사이의 거리가 멀 때 해당 정보를 이용하지 못하는 것을 말한다.
- 위 문제를 해결하기 위해서 transformer는 Recurrent Neural Network을 사용하지 않고 attention mechanism만 이용해서 dependency를 찾았다.

### Attention

- 어떤 결과를 만들어 내기 위해서 input data 어느 부분에 집중을 해야하는가?를 나타낼 수 있는 Model
  - 예를 들어) 영화 리뷰를 예측한다고 했을 때,
  - "이 영화 너무너무 재미있다. - 9"라는 input 문장이 있을 때, 9점이라는 점수를 얻기 위해서 '너무너무', '재미있다' 두 단어에 가중치가 높게 판단되어야 한다.

- Attention은 두 가지가 있다.
  - additive attention
    - dot-product attention과 다르게 scaling 과정이 없다.
  - dot-product attention
- 또한, 두 알고리즘의 차이는 시간이다. dot-product attention이 matrix multiplication에 대한 최적화가 많이 되어있기 때문에 더 빠르다.

## Computation

- 기존에 사용하던 Recurrent Neural Network의 경우에는 순차적인 계산이 필요하기 때문에 병렬적으로 계산을 할 수 없었습니다. 따라서, 계산 속도가 매우 느렸습니다.
- 하지만, transformer는 encoder 부분에서 attention으로 각각의 데이터 position에 대해서 attention만 주고,
- decoder 부분에서 masking을 해줌으로써 병렬적으로 계산을 할 수 있게 되었습니다.

## Model

- Encoder, encoder의 경우 input data의 representation을 구합니다.
- Decoder, decoder의 경우 encoder에서 요약한 representation을 통해서 output을 생성하게 됩니다.
- 또한, 각각의 step에서 다음 output 을 만들 때 이전에 만들어진 output을 이용한다.
  - 예를 들어) 나는 밥을 먹는다. 라는 문장에서 '나는'이 생성되었다면 '밥을'을 생성할 때 '나는'을 이용한다. 이것을 auto regressive하다고 한다.

- 또한, 아래의 Model 사진을 보면 Inputs와 Outputs(shifted right) 두 개가 나오는데, Inputs은 내가 입력으로 주는 데이터를 의미하고, Outputs(shifted right)는 위에서 언급한 것 처럼 output이 하나씩 나오게 되는데 이것을 넣는 것이다.

### Transformer architecture

![default](https://user-images.githubusercontent.com/22078438/51438156-896c9e00-1ceb-11e9-96f5-16f7a69793e7.PNG)


### Encoder

- input data가 주어지면, sub-layer, multi-head self-attention mechanism, position-wise fully connected feed-forward network을 지나게 된다.
- 이때, residual connection으로 각 layer input과 layer output을 이어준다. 이후 normalization을 해준다.

### Decoder

- encoder 결과에 multi-head attention mechanism을 수행할 sub-layer를 추가한다.
- 이후, residual connection과 normalization을 추가해준다.
- decoder에서는 순차적으로 결과를 만들어내야 하기 때문에 self-attention을 변형하여 masking을 해준다.
  - masking, 내가 만들고자 하는 단어의 위치를 \\(i\\)라 했을 때 \\(i\\)보다 뒤에 있는 단어를 보고 \\(i\\)번째 단어를 예측한다면 잘못된 것이기 때문에 이미 예측한 output을 이용해서 현재 위치 \\(i\\)의 단어를 만든다.

## Scaled Dot-Product Attention, Multi-Head Attention

![default](https://user-images.githubusercontent.com/22078438/51438345-524bbc00-1cee-11e9-993e-52ec818f1651.PNG)


### Scaled Dot-Product Attention

- input,
  - \\(d_k\\) 차원의 Q(query)와 K(key)
  - \\(d_v\\) 차원의 V(value)
- 먼저 Q와 K의 dot product를 계산하고, \\(\sqrt{d_k}\\)로 나눠주면 scaling 된다. 따라서 Scaled Dot-Product Attention이라 한다.
- 이후, softmax를 통과시켜 V에 대한 weights를 얻는다.<br><br>
$$ Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V $$

- K와 V는 attention의 위치와 관계없이 같은 값을 가진다.
- 그리고, Q와 K의 dot-product는 Q와 K사이의 유사도를 구하는 것이다.
- 따라서 cosine similarity를 구하고 softmax를 통과시킨 값을 V에 곱해주면 중요한 V일수록 더 높은 값을 가지게 된다.

- **Query(Q)는 각각의 query가 어떤 단어와 연관 되어 있는지 알기 위해서 모든 key(K)와 연산한다. 위 연산을 보면 query와 key가 dot-product하고 softmax를 취한다. 이것은 하나의 query가 모든 key와의 연관성을 계산하고, 그 값을 확률(softmax)로 바꿔준다. 이 결과로 query가 어떤 key와 얼마나 연관을 가지는지 알 수 있다.**
- **이렇게 구한 확률값을 value(V)에 곱해서 value에 대해서 scaling 한다.**
- **key와 value는 같은 단어를 의미하지만 두 개로 나뉜 것은 key는 각 단어와 연관성을 확률로 나타내고, value는 그 확률을 이용해서 attention 값을 계산하기 위해서 사용한다.**

### Multi-Head Attention

$$ MultiHead(Q,K,V) = Concat({head}_1, ..., {head}_h)W^O\;\;where\;\; {head}_i = Attention(QW^Q_i, KW^K_i, VW^V_i) $$

- \\(d_{model}\\) 차원의 K(key), V(value), Q(query)로 하나의 attention을 수행하는 대신에 K, V, Q에 각각 다르게 학습된 linear projection을 \\(h\\) 수행한다. 따라서, 동일한 K, V, Q에 각각 다른 weight matrix \\(W\\)를 곱해준다.
- 위에서 projection이라고 언급한 이유는 각각이 matrix와 곱해졌을 때 \\(d_k, d_v, d_{model}\\)차원으로 바뀌기 때문이다.
- 따라서 projection된 K, V, Q를 병렬적으로 attention mechanism을 취해서 \\(d_v\\) 차원의 output이 된다.
- 이후, \\(h\\)개의 \\(head\\)를 concatenate한 후 projection을 취하여 \\(d_{model}\\) 차원의 output이 된다.
- \\(d_Q, d_K, d_V\\)는 query, key, value의 개수

## Self-Attention

- Encoder 부분은 이전 layer에서 오는 key, value, query를 이용한다.

### Decoder

- encoder 부분과 다르게 Masked Multi-Head Attention을 사용한다.
- 위 Masking을 위해서<br><br>
$$ Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V $$<br>
- 위 식에서 \\(i\\)번째 이후 position은 softmax의 input값을 \\(-\infty\\)로 한다. 이렇게 함으로써 \\(i\\)번째 이후 position에 attention을 주는 경우가 없다.

### Encoder-Decoder Attention

- query는 decoder layer에서 오고, key와 value는 encoder layer output에서 온다. 그래서 encoder-decoder를 이용해서 query, key, value에 attention을 줄 수 있다.
- query가 decoder 부분에 있는 이유는 여기서 query는 조건으로 사용된다. 여기서 query의 의미는 현재 decoder 부분에서 특정 값이 주어졌을 때 다음 output이 무엇이 되어야 하는가? 이다.

## Position-wise Feed-Forward Networks

- encoder와 decoder 마지막 부분에 Feed-Forward Networks가 들어가있다.
- 이것이 Position-wise인 이유는 각각의 position마다 다르게 적용되기 때문이다.

- network architecture
  - Linear transformation
  - Activation, ReLU
  - Linear transformation<br><br>
  $$ FFN(x) = max(0, xW_1 + b_1)W_2 + b_2 $$

- 위 network에서 Linear transformation은 convolution filter가 1x1인 convolution 연산으로도 볼 수 있다.

## Positional Encoding

- Transformer는 Recurrent Neural Network와 달리 단어의 sequence를 잡을 수 없습니다. 따라서 단어 position마다 추가 정보를 주는 것이 필요하다.
- 그러므로 encoder, decoder에서 input embedding 부분에 positional encoding을 했다.
- positional encoding의 결과 차원은 embedding vector 차원과 같은 차원이기 때문에 더하기 가능하다.

- annotation
  - \\(pos\\), position
  - \\(i\\), 차원
  - 주기가 \\(10000^{2i / d_{model}} \cdot 2\pi\\)인 삼각함수
- transformer 논문에서는 다른 frequency를 가지는 sine과 cosine을 사용하였다.
  - frequency,
  - \\(PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})\\)
  - \\(PE_{(pos, 2i + 1)} = cos(pos / 10000^{2i / d_{model}})\\)

- 따라서, \\(pos\\)는 sequence에서 단어의 위치이고 각각의 단어는 \\(i\\)에 0부터 \\(\frac{d_{model}}{2}\\)까지 대입해서 \\(d_{model}\\)차원의 positional encoding vector를 얻는다.
- \\(k = 2i + 1\\) 일때는 cosine 함수, \\(k = 2i\\) 일때는 sine함수를 쓴다.
- 위와 같이 positional encoding vector를 \\(pos\\)마다 구하면 같은 column이라도 \\(pos\\)가 다르면 다른 값을 가지게 된다.
- 결론적으로, \\(pos\\)마다 다른 \\(pos\\)와 구분되는 positional encoding 값을 얻게 됨.

- 따라서 최종 식을 구하면,<br><br>
$$ PE_{pos}=[cos(pos/1), sin(pos/10000^{2/d_{model}}),cos(pos/10000)^{2/d_{model}},...,sin(pos/10000)] $$

- 위 식을 \\(PE_{pos + k}\\)는 \\(PE_{pos}\\)의 Linear function으로 나타내면,<br><br>
$$ PE_{(pos+k, 2i)}=sin(\frac{pos+k}{c})=sin(\frac{pos}{c})cos(\frac{k}{c})+cos(\frac{pos}{c})sin(\frac{k}{c}) =PE_{(pos,2i)}cos(\frac{k}{c})+cos(\frac{pos}{c})sin(\frac{k}{c}) $$<br>
$$ PE_{(pos+k, 2i+1)}=cos(\frac{pos+k}{c})=cos(\frac{pos}{c})cos(\frac{k}{c})-sin(\frac{pos}{c})sin(\frac{k}{c}) =PE_{(pos,2i+1)}cos(\frac{k}{c})-sin(\frac{pos}{c})sin(\frac{k}{c}) $$

## 왜 self-attention만 사용해서도 가능한가?

- layer마다 전체 연산량이 줄어든다.
- 앞서 언급한 것과 같이 Recurrent Neural Network과 달리 병렬화가 가능하기 때문에 연산을 빨리 할 수 있다.
- long-range term들의 dependency도 잘 학습 가능하다.
- 부가적으로, model 자체를 이해하기도 쉽다는 것이다.

# Reference

[참조 블로그](https://pozalabs.github.io/transformer/)<br>
[Transformers 참조 블로그](https://reniew.github.io/43/)

## 감사합니다.